In [36]:
## Travel time cell
from datetime import datetime, timedelta
from pytz import timezone as ptz

DATE_FORMAT = "%Y-%m-%dT%H:%M:%-SZ"
oneDay = timedelta(1)

departureTimeOptions = {
    'now': lambda tz : datetime.now(tz).replace(minute=59, second=59).isoformat(),
    'morning': lambda tz : (datetime.now().replace(hour=9, minute=0, second=0) + oneDay).isoformat(),
    'noon': lambda tz : (datetime.now(tz).replace(hour=12, minute=0, second=0) + oneDay).isoformat(),
    'afternoon': lambda tz : (datetime.now(tz).replace(hour=5, minute=0, second=0) + oneDay).isoformat(),
    'evening': lambda tz : (datetime.now(tz).replace(hour=7, minute=0, second=0) + oneDay).isoformat(),
}

print("now time: " + departureTimeOptions['now'](ptz('EST')))
print("morning time: " + departureTimeOptions['morning'](ptz('EST')))
print(datetime.now().timestamp())

now time: 2023-09-27T16:59:59.152712-05:00
morning time: 2023-09-28T09:00:00.153024
1695848923.153085


In [34]:
import requests
from const import GOOGLE_MAPS_API_KEY



COMPUTE_ROUTE_URL = 'https://routes.googleapis.com/directions/v2:computeRoutes'
SOME_HOUSE = '3025 The Credit Woodlands, Mississauga, ON L5C 2V3'
SQUARE_ONE_ADDRESS = '100 City Centre Dr, Mississauga, ON L5B 2C9'
# travelTimes = {
#     now: lambda : datetime.datetime.now()
#     morning: lambda : datetime.datetime
# }


def computeRoutes(originAddress: str, destinationAddress: str, departureOption = 'now'):

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
    }

    json_data = {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': False,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
        'languageCode': 'en-US',
        'units': 'IMPERIAL',
    }

    response = requests.post(
        COMPUTE_ROUTE_URL, headers=headers, json=json_data)

    return response.text


print(computeRoutes(SOME_HOUSE, SQUARE_ONE_ADDRESS))


{
  "routes": [
    {
      "distanceMeters": 6812,
      "duration": "991s",
      "polyline": {
        "encodedPolyline": "omyhGtlceNkCuD?Gf@{@LCXN|EtFXd@DRCXq@tAb@l@tA{EF]u@g@uAw@cB}AqBaCSIK?qCeDsCcDiDcE{@}@gFoGoBmC}GcIkEkFqCaDkCkDqJiL}@eAc@cAiJuKeDzEg@~@Of@qEnHyG|K_@Tw@fAkEbH}GzK]b@Kd@m@fA_KbPcC~DoCjEwF`JmAtAsAvAUHc@f@oDdFyCuDeBoBaGqGiLaNiFoGyCkDoRsU{LqOgIpOGRURGDgAD"
      }
    }
  ]
}

